In [ ]:
import cv2
import numpy as np
import threading
import tkinter as tk
import time
from tkinter import Label, Button, Frame, Canvas
from PIL import Image, ImageTk
from keras.models import model_from_json

# Loading the model
json_file = open("emotiondetectionmodel.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)

model.load_weights("emotiondetectionmodel.h5")

# Defining haarcascades for face recognition
haar_file = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(haar_file)

labels = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}
emotion_colors = {
    'angry': (0, 0, 255),      # Red
    'disgust': (0, 128, 0),    # Green
    'fear': (128, 0, 128),     # Purple
    'happy': (0, 255, 255),    # Yellow
    'neutral': (255, 255, 255), # White
    'sad': (128, 128, 0),      # Blue
    'surprise': (0, 255, 0)    # Light Green
}

prediction_active = True
current_emotion = "None"
face_detected_time = None
stabilization_period = 3
is_stabilizing = False
face_roi = None
confidence_score = 0
keypoints = [] 

def extract_features(image):
    feature = np.array(image)
    feature = feature.reshape(1, 48, 48, 1)
    return feature / 255.0

def end_application():
    root.quit()

def video_stream():
    global cap, label_text, current_emotion, face_roi, face_detected_time, is_stabilizing, confidence_score
    
    ret, frame = cap.read()
    
    if ret:
        frame = cv2.flip(frame, 1)
        original_frame = frame.copy()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        face_detected = False
        
        for (x, y, w, h) in faces:
            face_detected = True
            face_roi = gray[y:y+h, x:x+w]
            
            # Draw rectangle around face
            if is_stabilizing:
                # Create pulsing rectangle during stabilization
                pulse_value = int(255 * (np.sin(time.time() * 5) * 0.3 + 0.7))
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, pulse_value, pulse_value), 3)
                
                # Add progress bar for stabilization
                progress = min(1.0, (time.time() - face_detected_time) / stabilization_period)
                bar_width = int(w * progress)
                cv2.rectangle(frame, (x, y+h+5), (x+bar_width, y+h+15), (0, 255, 255), -1)
                cv2.rectangle(frame, (x, y+h+5), (x+w, y+h+15), (100, 100, 100), 1)
            else:
                # Draw emotion-colored rectangle after prediction
                if current_emotion in emotion_colors:
                    color = emotion_colors[current_emotion]
                    cv2.rectangle(frame, (x, y), (x+w, y+h), color, 4)
                else:
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            
            # Add visualization of facial keypoints 
            # For demonstration, we'll just show some sample keypoints on the right of the face
            if not is_stabilizing and current_emotion != "None":
                # Create a blue box for the keypoint visualization
                box_width = 150
                box_height = 150
                box_x = x + w + 10
                box_y = y
                
                # If there's no space on the right, place the box elsewhere
                if box_x + box_width > frame.shape[1]:
                    box_x = x - box_width - 10
                    if box_x < 0:
                        box_x = frame.shape[1] - box_width - 10
                        box_y = y + h + 10
                
                # Draw blue rectangle for keypoints visualization
                cv2.rectangle(frame, (box_x, box_y), (box_x + box_width, box_y + box_height), (255, 0, 0), 2)
                
                # Draw some keypoints and connections based on the emotion
                # This is just for demonstration - real facial landmarks would be different
                for i in range(5):
                    point_x = box_x + 30 + i * 20
                    point_y = box_y + 30
                    cv2.circle(frame, (point_x, point_y), 5, (0, 0, 255), -1)
                    
                    point_x2 = box_x + 30 + i * 20
                    point_y2 = box_y + 70
                    cv2.circle(frame, (point_x2, point_y2), 5, (0, 0, 255), -1)
                    
                    cv2.line(frame, (point_x, point_y), (point_x2, point_y2), (255, 255, 255), 1)
                    
                    if i < 4:
                        cv2.line(frame, (point_x, point_y), (point_x + 20, point_y), (255, 255, 255), 1)
                        cv2.line(frame, (point_x2, point_y2), (point_x2 + 20, point_y2), (255, 255, 255), 1)
            
            current_time = time.time()
            if face_detected_time is None:
                face_detected_time = current_time
                is_stabilizing = True
                label_text.set(f"Stabilizing: Please hold position for {stabilization_period} seconds")
            
            elapsed_time = current_time - face_detected_time
            remaining_time = max(0, stabilization_period - elapsed_time)
            
            if is_stabilizing:
                label_text.set(f"Stabilizing: Please hold position for {remaining_time:.1f} seconds")
                
                # Enhanced countdown text with background
                countdown_text = f"Hold for: {remaining_time:.1f}s"
                text_size = cv2.getTextSize(countdown_text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
                cv2.rectangle(frame, (x, y-30), (x + text_size[0] + 10, y-5), (0, 0, 0), -1)
                cv2.putText(frame, countdown_text, (x+5, y-10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
                
                if elapsed_time >= stabilization_period and prediction_active:
                    is_stabilizing = False
                    
                    try:
                        resized_face = cv2.resize(face_roi, (48, 48))
                        img = extract_features(resized_face)
                        pred = model.predict(img)
                        
                        predicted_idx = pred.argmax()
                        confidence_score = float(pred[0][predicted_idx])
                        prediction_label = labels[predicted_idx]
                        current_emotion = prediction_label
                        
                        # Update prediction text
                        label_text.set(f"Detected: {prediction_label.upper()} ({confidence_score:.2f})")
                    
                    except Exception as e:
                        print(f"Error processing face: {e}")
                        label_text.set(f"Error: {str(e)[:30]}...")
            else:
                if current_emotion != "None":
                    # Enhanced emotion display with background
                    result_text = f"{current_emotion.upper()} ({confidence_score:.2f})"
                    text_size = cv2.getTextSize(result_text, cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, 2)[0]
                    text_bg_color = emotion_colors.get(current_emotion, (255, 0, 0))
                    cv2.rectangle(frame, (x, y-40), (x + text_size[0] + 10, y-5), (0, 0, 0), -1)
                    cv2.putText(frame, result_text, (x+5, y-15), 
                               cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, text_bg_color, 2)
        
        if not face_detected:
            face_detected_time = None
            is_stabilizing = False
            if prediction_active:
                label_text.set("No face detected")
                current_emotion = "None"
        
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame_rgb)
        imgtk = ImageTk.PhotoImage(image=img)
        lbl.imgtk = imgtk
        lbl.configure(image=imgtk)
    
    root.after(10, video_stream)

# Create the main window
root = tk.Tk()
root.title("Emotion Recognition")
root.configure(bg="#1e56a0")  # Royal blue background 

# Set window size
window_width = 1200
window_height = 700
screen_width = root.winfo_screenwidth()
screen_height = root.winfo_screenheight()
x = (screen_width - window_width) // 2
y = (screen_height - window_height) // 2
root.geometry(f"{window_width}x{window_height}+{x}+{y}")

# Create a two-column layout
main_frame = Frame(root, bg="#1e56a0")
main_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)

# Left column for instructions
left_column = Frame(main_frame, bg="#1e56a0", width=400)
left_column.pack(side=tk.LEFT, fill=tk.BOTH, padx=(0, 10))
left_column.pack_propagate(False)  # Prevent shrinking

# App title
title_label = Label(left_column, text="Expression Recognition", 
                   font=("Arial", 28, "bold"), bg="#1e56a0", fg="white")
title_label.pack(pady=(10, 20))

subtitle_label = Label(left_column, text="How to use EmotionWave", 
                      font=("Arial", 22, "bold"), bg="#1e56a0", fg="white")
subtitle_label.pack(pady=(0, 20))

# Instructions 
instructions = [
    "1. Position Your Face inside the frame located at the top right of your screen.",
    "2. Ensure good lighting conditions for better recognition accuracy.",
    "3. Maintain an ideal distance from your camera (similar to your regular sitting position with a laptop).",
    "4. Show the Facial Expression that you would like the system to predict.",
    "5. Hold Your Face Steady for 5 seconds while the system stabilizes.",
    "6. The system will predict your emotion after the stabilization period.",
    "7. If the prediction is incorrect, please click the Incorrect Button to report it.",
    "8. To finish, press the End Button to return to the previous screen."
]

for instr in instructions:
    instruction_label = Label(left_column, text=instr, justify=tk.LEFT, wraplength=380,
                            font=("Arial", 14), bg="#1e56a0", fg="white")
    instruction_label.pack(anchor=tk.W, pady=8)

# Right column for video and prediction
right_column = Frame(main_frame, bg="#1e56a0")
right_column.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

# Video display area
video_frame = Frame(right_column, bg="#1e56a0", bd=2, relief=tk.RIDGE)
video_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

lbl = Label(video_frame, bg="black")
lbl.pack(padx=5, pady=5, fill=tk.BOTH, expand=True)

# Prediction output section
prediction_frame = Frame(right_column, bg="#1e56a0", height=100)
prediction_frame.pack(fill=tk.X, padx=10, pady=10)

prediction_label = Label(prediction_frame, text="Predicted Output:", 
                        font=("Arial", 18, "bold"), bg="#1e56a0", fg="white")
prediction_label.pack(pady=(10, 5))

# Status text box with white background for prediction
label_text = tk.StringVar()
label_text.set("Waiting for face detection...")
label = Label(prediction_frame, textvariable=label_text, font=("Arial", 16), 
             bg="white", fg="black", width=30, height=2)
label.pack(pady=5)

# Button frame at bottom
button_frame = Frame(right_column, bg="#1e56a0")
button_frame.pack(fill=tk.X, padx=10, pady=10)

# End button
end_btn = Button(button_frame, text="END", font=("Arial", 14, "bold"),
                width=10, height=1, bg="#e94560", fg="white", command=end_application)
end_btn.pack(side=tk.RIGHT, padx=5, pady=10)

# Initialize webcam
cap = cv2.VideoCapture(0)

# Start video stream
video_stream()

# Keyboard shortcuts
def exit_program(event):
    root.quit()

root.bind("<KeyPress-q>", exit_program)  

# Start the main loop
root.mainloop()

# Clean up
cap.release()
cv2.destroyAllWindows()